In [83]:
import os
import spacy
import pandas as pd
from spacy.tokens import DocBin


FILE = 'B2W-Reviews01.csv'

PATH = os.path.join(os.path.abspath(''), FILE)

df = pd.read_csv(PATH)

#mostrar quais colunas existem no dataset
print(df.columns)

#manter apenas as colunnas  review_text e overall_rating
df = df[['review_text', 'overall_rating']]

#renomear as colunas para text e sentiment
df.columns = ['text', 'sentiment']

#trocar os valores quando for 1 e 2 sera negative, 3 sera neutral  e 4 e 5 sera positive
df['sentiment'] = df['sentiment'].apply(lambda x: 'negative' if x < 3 else 'neutral' if x == 3 else 'positive')

#Conta a frequência de cada valor único presente na coluna 'sentiment'
df['sentiment'].value_counts()

Index(['submission_date', 'reviewer_id', 'product_id', 'product_name',
       'product_brand', 'site_category_lv1', 'site_category_lv2',
       'review_title', 'overall_rating', 'recommend_to_a_friend',
       'review_text', 'reviewer_birth_year', 'reviewer_gender',
       'reviewer_state'],
      dtype='object')


C:\Users\samir\AppData\Local\Temp\ipykernel_23208\1130710681.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH)


sentiment
positive    80300
negative    35758
neutral     16315
Name: count, dtype: int64

In [84]:
df

,text,sentiment
0,Estou contente com a compra entrega rápida o ú...,positive
1,"Por apenas R$1994.20,eu consegui comprar esse ...",positive
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,positive
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,positive
4,"A entrega foi no prazo, as americanas estão de...",positive
...,...,...
132368,"Vale muito, estou usando no controle do Xbox e...",positive
132369,"Prático e barato, super indico o produto para ...",positive
132370,Chegou antes do prazo previsto e corresponde a...,positive
132371,"Material fraco, poderia ser melhor. Ficou deve...",negative


In [85]:
# diminuir o tamanho do dataset para 8361 mas manter a proporção de sentimentos, se tiver text nulo remover
df = df.dropna(subset=['text'])
df = df.groupby('sentiment').apply(lambda x: x.sample(8361 // 3)).reset_index(drop=True)

df['sentiment'].value_counts()

# #diminuir o tamanho do dataset para 8361, e nao deixar registros text nulo
# df = df.dropna(subset=['text'])
# df = df.sample(n=8361, random_state=1)


C:\Users\samir\AppData\Local\Temp\ipykernel_23208\3849543489.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('sentiment').apply(lambda x: x.sample(8361 // 3)).reset_index(drop=True)


sentiment
negative    2787
neutral     2787
positive    2787
Name: count, dtype: int64

In [86]:
#verificar se a coluna text possui valores nulos
df['text'].isnull().sum()
df['sentiment'].value_counts()

sentiment
negative    2787
neutral     2787
positive    2787
Name: count, dtype: int64

O DataFrame df é agrupado pela coluna sentiment, que contém os rótulos de sentimento (por exemplo, 'positive', 'negative', 'neutral').
Para garantir que o conjunto de treino seja balanceado (ou seja, tenha um número igual de instâncias de cada classe de sentimento), o código amostra um número igual de instâncias de cada grupo.
O tamanho da amostra é definido como o tamanho mínimo dos grupos menos 49. Isso é feito para evitar que o grupo menor seja subamostrado demais.
O parâmetro random_state=25 é usado para garantir que a amostragem seja reprodutível, ou seja, que a mesma amostra seja obtida toda vez que o código for executado.

In [87]:
train = df.groupby('sentiment').sample(df.groupby('sentiment').size().min()-800, random_state = 25)
test = df.drop(train.index)

print(test.index)
print(train.index)

print(train.shape, test.shape)

print(train['sentiment'].value_counts())
print(test['sentiment'].value_counts())

Index([   0,    6,   10,   11,   14,   20,   22,   23,   25,   26,
       ...
       8331, 8332, 8336, 8340, 8341, 8342, 8349, 8355, 8357, 8359],
      dtype='int64', length=2400)
Index([1527,  999, 1400,  616, 1868,  598,  899, 1367, 2321,  894,
       ...
       5892, 8003, 8347, 7244, 6573, 5993, 8299, 7638, 8175, 6284],
      dtype='int64', length=5961)
(5961, 2) (2400, 2)
sentiment
negative    1987
neutral     1987
positive    1987
Name: count, dtype: int64
sentiment
negative    800
neutral     800
positive    800
Name: count, dtype: int64


In [88]:
#calcular a porcentagem total de quantidade para treino e teste, sem considerar a coluna 'sentiment'
print('Treino:', train.shape[0]/df.shape[0])
print('Teste:', test.shape[0]/df.shape[0])

Treino: 0.7129529960531037
Teste: 0.2870470039468963


In [89]:
# transforma os DataFrames train e test em listas de tuplas, onde cada tupla contém o texto e o sentimento correspondente
train['tuples'] = train.apply(lambda row: (row['text'],row['sentiment']), axis=1)
train = train['tuples'].tolist()
test['tuples'] = test.apply(lambda row: (row['text'],row['sentiment']), axis=1)
test = test['tuples'].tolist()
train[0]

('Recebi somente um cartucho colorido, os outros a empresa parceira e as Americanas não responderam a minha queixa.  Devido a essa situação, não usei o cartucho, pois não sei se farei a devolução.',
 'negative')

In [90]:
# User function for converting the train and test dataset into spaCy document

nlp=spacy.load("pt_core_news_lg")
def document(data):
#Creating empty list called "text"
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 0
    elif (label=='negative'):
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
      doc.cats['neutral']  = 0
    else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 1
    #Adding the doc into the list 'text'
    text.append(doc)
  return(text)

In [91]:
#passing the train dataset into function 'document'
train_docs = document(train)
test_docs = document(test)

#Creating binary document using DocBin function in spaCy
doc_bin_train = DocBin(docs = train_docs)
doc_bin_test = DocBin(docs = test_docs)

#Saving the binary document as train.spacy
doc_bin_train.to_disk("train.spacy")
doc_bin_test.to_disk("test.spacy")